In [ ]:
%load_ext autoreload
%autoreload 2
# Packages that we are going to use in this project
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


import sys 
import os

import matplotlib.pyplot as plt
import plotly.express as px

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf

from scipy.fftpack import fft
from scipy.signal import blackman
from scipy.signal import periodogram

from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.preprocessing import MinMaxScaler

from functions import *


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\df")
from manage_file import FILES, getPath

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


## STEP 1: Read & view of the data

In [ ]:
df = read_data(file="minable")

In [ ]:
df.to_csv("asd.csv")

In [ ]:
# READ DATA INPUT : earth, moon or minable. Prefer use minable, 
# that contains earth and moon data, with clustering and interpolation already done
df = read_data(file="minable")
df = df[df.mag > 4.4]
df.head(10)
len(df)

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
df1 = filter_dataframe(df,var="dec", var_range = (-29,29), mag_range=(4.8,10))
df1.head(10)

## Filter dataframe

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
#df1 = filter_dataframe(df,var="ill_frac_interpolated", var_range = (40,60), mag_range=(5,10))
#df1

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
df2 = filter_dataframe(df1,var="ra/h", var_range = (1,3), mag_range=(4.8,10))
df2.head(10)

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
df3 = filter_dataframe(df2,var="dec", var_range = (-2,2), mag_range=(4.8,10))
df3.head(100)

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
df4 = filter_dataframe(df3,var="depth", var_range = (200,700), mag_range=(4.8,10))
df4.head(100)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df3, x="dec",color='ra/h', barmode='group',height=400)

In [ ]:
df = df.sort_index()
#df_moon.head (20)
print (df [13940:13960])

## STEP 2:  __Frequency for a period of time of earthquakes considering magnitude segmentation.__

For ex:
* period_length = 10 (every 10 years compare)
* mag_seg = {magnitude < 4 |  4<= magnitude < 5 | 5 < magnitude }

In [ ]:
######## you must want to change these piece of code ########
period_length = 5
#############################################################
df1 = period_calculation(df1, period_length)# how many earthquakes for each period
# in case you dont want to use the filtered dataframe change df1 for df.

In [ ]:
######## you must want to change these piece of code ########
##### also you can not use “/ “ to describe the variable segment, python does not like it, but you must use it as the variable itself , as listed in the index,  
#['time', 'year', 'month', 'day', 'latitude', 'longitude', 'mag', 'depth',
#       'Pais', 'PERIOD', 'MAG_SEG', 'NewDate', 'NewMonth', 'cluster_label',
#       'acum_day', 'ill_frac', 'r/km', 'dec', 'ra/h', 'ra/°',
#       'ill_frac_interpolated', 'r/km_interpolated', 'dec_interpolated',
#       'td_acc', 'td_acc_interpolated'] #######

#mag_seg = {0: [4.5,4.8], 1: [4.8,5.1], 2: [5.1,5.4], 3:[5.4,5.7], 4:[5.7,6], 5:[6,6.6], 6:[6.6,7.5], 7:[7.5,10] }
#dec_seg = {0: [-29,-18], 1: [-18,-9], 2: [-9,0], 3:[0,9], 4:[9,18], 5:[18,29]}
#depth_seg = {0: [0,20], 1: [20,40], 2: [40,100], 3:[100,200], 4:[200,400], 5:[400,800]}
#latitude_seg = {0: [-80,-50], 1: [-50,-28], 2: [-28,-18], 3:[-18,0], 4:[0,18], 5:[18,28], 6:[28,50], 7:[50,80] }
#longitude_seg = {0: [-180,-135], 1: [-135,-90], 2: [-90,-45], 3:[-45,0], 4:[0,45], 5:[45,90], 6:[90,135], 7:[135,180] }
#ill_frac_seg = {0: [0,20], 1: [20,40], 2: [40,60], 3:[60,80], 4:[80,100]}
#rah_seg = {0: [0,3], 1: [3,6], 2: [6,9], 3:[9,12], 4:[12,15], 5:[15,18], 6:[18,21], 7:[21,24] }
#rkm_seg = {0: [350000,360000], 1: [360000,370000], 2: [370000,380000], 3:[380000,390000], 4:[390000,400000], 5:[400000,402000], 6:[402000,410000]}
#td_acc_seg = {0: [0.3,0.7], 1: [0.7,1], 2: [1,1.3], 3:[1.3,1.6], 4:[1.6,2]}

##acum_day_seg = {0: [0,91], 1:[91,182], 2:[182,273], 3: [273,366] }
acum_day_seg = {0: [355,20], 1:[20,50], 2:[50,81], 3: [81,111], 4:[111,141], 5: [141,172], 6:[172,202], 7: [202,232], 8: [232,263], 9: [263,293], 10: [293,324], 11:[324,355] }
mag_seg = {0: [4.5,4.8], 1: [4.8,5.1], 2: [5.1,5.4], 3:[5.4,5.7], 4:[5.7,6], 5:[6,6.6], 6:[6.6,7.5], 7:[7.5,7.9], 8:[8,10] }
#mag_seg = {0: [6.99,9.5]}
#dec_seg = {0: [-29,-18], 1: [-18,-4], 2: [-4,4], 3:[4,18], 4:[18,29]}
#dec_seg = {0: [-29,-18], 1: [-18,-0], 2: [0,18], 3:[18,129]}
dec_seg = {0: [-29,-18.134], 1: [-18.134,-9.0], 2:[-9,-3], 3:[-3,3], 4: [3,9], 5: [9,18.134], 6:[18.134,29] }
##dec_seg = {0: [-29,0], 1: [0,29]}
#dec_seg = {0: [-7,-6], 1: [-6,-5], 2: [-5,-4], 3: [-4,-3], 4: [-3,-2], 5: [-2,-1], 6:[-1,0], 7: [0,1], 8: [1,2], 9: [2,3], 10: [3,4], 11: [4,5], 12: [5,6], 13: [6,7] }
##dec_seg = {0: [-29,29]}
##depth_seg = {0: [0,20], 1: [20,40], 2: [40,100], 3:[100,200], 4:[200,400], 5:[400,800]}
depth_seg = {0: [0,20], 1: [20,40], 2: [40,100], 3:[100,200], 4:[200,300], 5:[300,400], 6:[400,500], 7:[500,600], 8:[600,700], 9:[700,800]}
latitude_seg = {0: [-90,-50], 1: [-50,-28], 2: [-28,-18], 3:[-18,0], 4:[0,18], 5:[18,28], 6:[28,50], 7:[50,90] }
longitude_seg = {0: [-180,-135], 1: [-135,-90], 2: [-90,-45], 3:[-45,0], 4:[0,45], 5:[45,90], 6:[90,135], 7:[135,180] }
##ill_frac_seg = {0: [0,10], 1: [10,27.5], 2: [27.5,45], 3:[45,55], 4:[55,72.5], 5:[72.5,90], 6:[90,100]}
ill_frac_seg = {0: [0,10], 1: [10,22], 2: [22,33], 3:[33,45], 4:[45,55], 5:[55,67], 6:[67,78], 7:[78,90], 8:[90,100]}
##ill_frac_seg = {0: [0,10], 1: [10,40], 2: [40,60], 3:[60,90], 4:[90,100]}
##rah_seg = {0: [0,3], 1: [3,6], 2: [6,9], 3:[9,12], 4:[12,15], 5:[15,18], 6:[18,21], 7:[21,24] }
rah_seg = {0: [23,1], 1: [1,3], 2: [3,5], 3:[5,7], 4:[7,9], 5:[9,11], 6:[11,13], 7:[13,15], 8:[15,17], 9:[17,19], 10:[19,21], 11:[21,23] }
##rad_seg = {0: [0,30], 1: [30,60], 2: [60,90], 3:[90,120], 4:[120,150], 5:[150,180], 6:[180,210], 7:[210,240], 8:[240,270], 9:[270,300], 10:[300,330], 11:[330,360] }
rad_seg = {0: [0,15], 1: [15,30], 2: [30,45], 3:[45,60], 4:[60,75], 5:[75,90], 6:[90,105], 7:[105,120], 8:[120,135], 9:[135,150], 10:[150,165], 11:[165,180], 12:[180,195], 13:[195,210], 14:[210,225], 15:[225,240], 16:[240,255], 17:[255,270], 18:[270,285], 19:[285,300], 20:[300,315], 21:[315,330], 22:[330,345], 23:[345,360] }
##rkm_seg = {0: [350000,360000], 1: [360000,370000], 2: [370000,380000], 3:[380000,390000], 4:[390000,400000], 5:[400000,410000]}
rkm_seg = {0: [356000,358000], 1: [358000,360000], 2:[360000,362000], 3: [362000,364000], 4:[364000,366000], 5: [366000,368000], 6:[368000,370000], 7: [370000,372000], 8: [372000,374000], 9:[374000,376000], 10: [376000,378000], 11:[378000,380000], 12: [380000,382000], 13:[382000,384000], 14:[384000,386000], 15:[386000,388000], 16:[388000,390000], 17:[390000,392000], 18:[392000,394000], 19:[394000,396000], 20:[396000,398000], 21:[398000,400000], 22:[400000,402000], 23:[402000,404000], 24:[404000,405000], 25:[405000,406000], 26:[406000,406500], 27:[406500,407000] }
##rkm_seg = {0: [350000,410000]}
##rkm_seg = {0: [403000,403500], 1: [403500,404000], 2:[404000,404500], 3: [405000,405500], 4:[405500,406000], 5: [406000,406500], 6:[406500,407000] }
##rkm_interpolated_seg = {0: [400000,403500], 1: [403500,404000], 2:[404000,404500], 3: [405000,405500], 4:[405500,406000], 5: [406000,406500], 6:[406500,407000] }
##rkm_seg = {0: [370000,374000], 1: [374000,378000], 2:[378000,382000], 3: [382000,386000], 4:[386000,390000], 5: [390000,394000], 6:[394000,398000], 7:[398000,402000] }
##rkm_seg = {0: [356000,369700], 1: [369700,376000], 2:[376000,382000], 3: [382000,388000], 4: [388000,395000], 5:[395000,402000], 6: [402000,407000] }
##rkm_seg = {0: [356000,357000], 1: [357000,360000], 2:[360000,362000], 3: [362000,364000], 4:[364000,366000], 5: [366000,368000], 6:[368000,370000] }
##rkm_seg = {0: [356000,358000], 1: [358000,360000], 2:[360000,362000], 3: [362000,364000], 4:[364000,366000], 5: [366000,368000], 6:[368000,370000], 7: [370000,374000], 8: [374000,378000], 9:[378000,382000], 10: [382000,386000], 11:[386000,390000], 12: [390000,394000], 13:[394000,398000], 14:[398000,402000], 15:[402000,404000], 16:[404000,405000], 17:[405000,406000], 18:[406000,406500], 19: [406500,407000] }
td_acc_seg = {0: [0.3,0.8], 1: [0.8,1], 2: [1,1.4], 3:[1.4,1.6], 4:[1.6,2]}


segmentations = {"mag": mag_seg, "dec": dec_seg, "dec_interpolated": dec_seg, "depth":depth_seg, "acum_day":acum_day_seg, "latitude": latitude_seg, "longitude": longitude_seg,
                "ill_frac": ill_frac_seg, "ill_frac_interpolated": ill_frac_seg, "ra/°": rad_seg, "ra/h": rah_seg, "r/km": rkm_seg, "r/km_interpolated": rkm_seg, "td_acc_interpolated": td_acc_seg, "td_acc": td_acc_seg}

#############################################################
for key, value in segmentations.items():
    df1 = variable_segmentation(df1, var_seg=value, var=key)


In [ ]:
df1["ILL_FRAC_SEG"].value_counts()

In [ ]:
50628/len(df1)

In [ ]:
df.columns

## Histograms Plots

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
px_histogram(df1, x="RA/°_SEG", color='DEC_SEG', barmode='stack', height=400)


In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
px_histogram(df2, x="RA/°_SEG", color='DEC_SEG', barmode='stack', height=400)


In [ ]:
distribution_plot(df1, var="ra/h", specific_cluster=None)
##df.head(10)

In [ ]:
df1["RA/H_SEG"].value_counts(5)
#len(df)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
px_histogram(df1, x="DEC_SEG", color='RA/H_SEG', barmode='group', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
px_histogram(df1, x="RA/H_SEG", color='DEC_SEG', barmode='stack', height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="PERIOD",color='DEC_SEG', barmode='stack',height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="ACUM_DAY_SEG", color= 'DEC_SEG', barmode='stack', height=400)


In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="RA/H_SEG", color= 'ACUM_DAY_SEG', barmode='stack', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="ILL_FRAC_SEG", color= 'RA/H_SEG', barmode='stack', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="ACUM_DAY_SEG", color= 'RA/H_SEG', barmode='stack', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="RA/H_SEG", color= 'ACUM_DAY_SEG', barmode='stack', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="ILL_FRAC_SEG", color= 'ACUM_DAY_SEG', barmode='stack', height=400)

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods and alot more
px_histogram(df1, x="ACUM_DAY_SEG", color= 'ILL_FRAC_SEG', barmode='stack', height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='R/KM_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="RA/°_SEG",color='ILL_FRAC_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="RA/°_SEG",color='ILL_FRAC_INTERPOLATED_SEG', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='RA/°_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='RA/°_SEG', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='RA/H_SEG', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="TD_ACC_INTERPOLATED_SEG",color='ILL_FRAC_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='TD_ACC_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="R/KM_INTERPOLATED_SEG",color='TD_ACC_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="TD_ACC_INTERPOLATED_SEG",color='R/KM_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='R/KM_INTERPOLATED_SEG', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="R/KM_INTERPOLATED_SEG",color='ILL_FRAC_INTERPOLATED_SEG', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="R/KM_INTERPOLATED_SEG",color='ILL_FRAC_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_INTERPOLATED_SEG",color='R/KM_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="DEPTH_SEG",color='cluster_label', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_SEG",color='MAG_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_SEG",color='cluster_label', barmode='group',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="cluster_label",color='ILL_FRAC_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="ILL_FRAC_SEG",color='DEPTH_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="LATITUDE_SEG",color='TD_ACC_INTERPOLATED_SEG', barmode='stack',height=400)

In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df1, x="PERIOD",color='DEC_SEG', barmode='stack',height=400)

In [ ]:
# Histogram groupby monthly
histogram_monthly(df1, date_off_set=False, bool_mag_seg=False)

In [ ]:
plot_monthly(df1,years=(1973,1978))

## Increasing by country

In [ ]:
# Count amount of earthquakes of each country
countries_value_counts(df1)

In [ ]:
######## you must want to change these piece of code ########
countries = ["Indonesia", "Japan", "Chile", "No_Country"]
#############################################################
histogram_countries(df1, countries)

### Plot map "No_country" 

In [ ]:
df1 = df[df.Pais == "No_Country"]
plot_map(df1, clusters=True)
# punto de extraccion de la data de la luna.

### Trendline for magnitude segments

In [ ]:
trendline_calculations(df1)

### Clustering

In [ ]:
#df = calculate_clustering(df, normalized=False)
df1.cluster_label.value_counts()

In [ ]:
df.columns

In [ ]:
plot_map(df1, clusters=True)

In [ ]:
plot_map(df3, clusters=True)

In [ ]:
series_trend, negative_trend = trendline_calculations(df1,cluster=True)

In [ ]:
# Plot map on negative trendline cluster
######## you must want to change these piece of code ########
clusters_toplot = list(negative_trend.keys()) # [48,8,46,18] for example
#clusters_toplot = [3,14]
#############################################################
plot_map(df1, clusters=True, specific_clusters=clusters_toplot)

In [ ]:
plot_map_animation(df3, specific_clusters=[], animation_frame="year")

In [ ]:
specific_cluster_info(df3, 3 )

In [ ]:
# histogram by specific clusters
# df3.cluster_label.unique().tolist()
clusters = df3.cluster_label.unique().tolist()
##histogram_cluster(df3,clusters)
px_histogram(df3, x="dec",color='cluster_label', barmode='group',height=400)
##df1["specific_cluster_info"].value_counts()

In [ ]:
# histogram by specific clusters
clusters = [47,43,16,37,41,5]
histogram_cluster(df3,clusters)

In [ ]:
histogram_overtime(df1, var="ill_frac_interpolated", specific_cluster=[37,5], animation_frame="year")
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

## Moon correlation

### Calculate distribution of r/km and ill_frac of the moon interpolated in earthquakes

In [ ]:
"""
Parameters:
        df (pandas.DataFrame): A pandas DataFrame object.
        columns (list): list of columns.
        step_quantile (float): The step between each quantile to compute. Default is 0.25.
        clusters (list): list of clusters of interest. [] by default.
"""


describe_columns(df3, ["ra/h", "r/km_interpolated", "ill_frac_interpolated", "dec_interpolated", "td_acc", "acum_day"],
                 step_quantile=0.01, clusters=[]) #clusters= list(negative_trend.keys())


In [ ]:
calculo_distribution(df1,var = "dec_interpolated", num_bins=100, specific_cluster=None)

In [ ]:
distribution_plot(df1, var="dec_interpolated", specific_cluster=None)
##df.head(10)

In [ ]:
distribution_plot(df1, var="dec", specific_cluster=None)
##df.head(10)

In [ ]:
distribution_plot(df1, var="acum_day", specific_cluster=None)
df1.acum_day.value_counts()

In [ ]:
distribution_plot(df1, var="year", specific_cluster=None)
df1.year.value_counts()

In [ ]:
distribution_plot(df1, var="ill_frac", specific_cluster=None)

In [ ]:
distribution_plot(df1, var="ACUM_DAY_SEG", specific_cluster=None)

In [ ]:
df1["ACUM_DAY_SEG"].value_counts()

In [ ]:
50628/len(df1)


In [ ]:
df1.head(10)

In [ ]:
plot_calculo_distribution(df1, var="DEPTH_SEG", histnorm="percent", num_bins=10, specific_cluster=None)

In [ ]:
df.columns

In [ ]:
plot_calculo_distribution(df1, var="depth", histnorm="", num_bins=60, specific_cluster=None)

In [ ]:
histogram_animation(df1, var="depth", histnorm="",animation_frame="year", nbins=200, 
                   range_x=[0,700], range_y=[0,2000] )
# nbins if you want by default write None.
# range_x and range_y by default None, and will be like it was before, if you want to change set a list with two numbers
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

In [ ]:
histogram_animation(df, var="mag", histnorm="",animation_frame="year", nbins=50, 
                   range_x=[4.4,10], range_y=[0,2200] )
# nbins if you want by default write None.
# range_x and range_y by default None, and will be like it was before, if you want to change set a list with two numbers
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

In [ ]:
histogram_animation(df, var="dec", histnorm="",animation_frame="year", nbins=100, 
                   range_x=[-29,29], range_y=[0,600] )
# nbins if you want by default write None.
# range_x and range_y by default None, and will be like it was before, if you want to change set a list with two numbers
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

In [ ]:
count_number_days(var="ill_frac", range_var=[0,100])

In [ ]:
# Distribution KDE plot for a varaible for earthquake data and moon data.
df_moon = read_data(file="moon")
distribution_plot(df, df_moon, var="td_acc")

In [ ]:
# date_filt is in the order "YYYY-mm-dd"
df3 = filter_by_date(df1, date_filt="2011-02-25", nweeks=4)

In [ ]:
plot_map_animation(df3,animation_frame="day")

In [ ]:
describe_columns(df1, ["r/km_interpolated", "ill_frac_interpolated", "td_acc_interpolated","dec_interpolated", "acum_day"],
                 step_quantile=0.05, clusters=[])

# Now in df_all is stored all the previous information and that of tidal accel.
#You can use all the previous functions and the new column is called td_acc.
# Then I'll make everything inside df itself.

In [ ]:
distribution_plot(df1, var="td_acc_interpolated", specific_cluster=None)